# Secure Boosting Tree 

## 資料品質檢查

### 設定資料路徑 & 參數

In [1]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "titanic_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/titanic_hetero_guest.csv")

### 缺失值 & 欄位名

#### 欄位名請全部調整成小寫
#### Age欄位有缺失值, 等等可以使用 DataTransform 來填平均年齡 29.7

In [2]:
import pandas as pd
dense_df = pd.read_csv(dense_data_dir)
print(dense_df.isna().sum())
print(dense_df.head(5))
print(f"The Average Age of Passenger is {dense_df.age.mean():.2f}")

passengerid      0
survived         0
pclass           0
sex              0
age            177
sibsp            0
dtype: int64
   passengerid  survived  pclass     sex   age  sibsp
0            1         0       3    male  22.0      1
1            2         1       1  female  38.0      1
2            3         1       3  female  26.0      0
3            4         1       1  female  35.0      1
4            5         0       3    male  35.0      0
The Average Age of Passenger is 29.70


## 上傳資料

In [3]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-06-06 06:37:45.655 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306060637455117550

2023-06-06 06:37:45.662 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


m2023-06-06 06:37:47.687 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-06 06:37:47.688 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-06 06:37:48.704 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-06 06:37:49.722 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-06 06:37:50.736 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-06-06 06:37:51.760 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-06-06 06:37:52.775 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:07
2023-06-06 

### 建構 Training pipeline 範例

使用 `Reader` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 Reader`

In [4]:
from pipeline.component import Reader
reader_0 = Reader(name="reader_0")

# set guest parameter
reader_0.get_party_instance(role='guest', party_id=guest).component_param(
    table={"name": "titanic_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=host).component_param(
    table={"name": "titanic_hetero_host", "namespace": "experiment"})

使用 `DataTransform` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 DataTransform`

`DataTransform` 負責資料前處理( 設定目標欄位名稱, 補缺值, 替換 outliers )

同時也可以設定各欄位的資料格式, `float`, `str`, `int` 等等

In [5]:
from pipeline.component import DataTransform
data_transform_0 = DataTransform(name="data_transform_0")

# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(
    with_label=True, label_name="survived", label_type="int",
    missing_fill=True, missing_fill_method="designated", default_value=["O", "male", 29.7, 0], outlier_replace=False,
    data_type="float", exclusive_data_type={"pclass":"str", "sex":"str", "sibsp":"str"}
)

data_transform_0.get_party_instance(role='host', party_id=[host]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=["100", "0.0", "N"], outlier_replace=False,
    data_type="float", exclusive_data_type={"parch":"str", "embarked":"str"}
)

In [6]:
from pipeline.component import OneHotEncoder

onehot_encoder_0 = OneHotEncoder(name="onehot_encoder_0")

onehot_encoder_0.get_party_instance(role='guest', party_id=guest).component_param(
    transform_col_indexes=[0, 1, 3], transform_col_names=["pclass", "sex", "sibsp"]
)

onehot_encoder_0.get_party_instance(role='host', party_id=[host]).component_param(
    transform_col_indexes=[0, 2], transform_col_names=["parch", "embarked"]
)

使用 `Intersection` 模塊來達到對齊雙方相同顧客 ID, 且不會洩漏非相同顧客的 ID

In [7]:
from pipeline.component import Intersection
intersect_0 = Intersection(name="intersect_0")

現在使用 `HeteroSecureBoost` 模塊. 用以下的參數來構建樹模型

In [8]:
from pipeline.component import HeteroSecureBoost
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


最後, 為了檢驗好壞 使用 `Evaluation` 模塊來驗證好壞

In [9]:
from pipeline.component import Evaluation
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

使用 `pipeline` 模塊來構建聯邦學習流程
實例化 `pipeline` 並設定 `initiator` 和 `roles`:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
        
上一個 component 的 output 是下一個 component 的 input

    - data_transform_0 吃 reader_0 的 output
    - intersect_0 吃 data_transform_0 的 output
    - hetero_secureboost_0 吃 intersect_0 的 output
    - evaluation_0 吃 hetero_secureboost_0 的 output (預測值)

記得用 `pipeline.compile()` 來打包整串流程
使用 `pipeline.fit()` 來開始進行訓練

In [10]:
from pipeline.backend.pipeline import PipeLine
from pipeline.interface import Data
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=guest) \
        .set_roles(guest=guest, host=host)

pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(onehot_encoder_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=onehot_encoder_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-06 06:37:57.266 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306060637569089540

2023-06-06 06:37:57.274 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-06-06 06:37:58.295 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-06 06:37:58.296 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:01
2023-06-06 06:37:59.311 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-06-06 06:38:00.328 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-06-06 06:38:01.385 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-06-06 06:38:02.402 | INFO     

2023-06-06 06:38:35.926 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-06 06:38:37.085 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-06 06:38:38.104 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-06 06:38:39.127 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
2023-06-06 06:38:40.143 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:42
2023-06-06 06:38:41.215 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:43
2023-06-06 06:38:42.335 | INFO     | pipeline.utils.invoker.job_

2023-06-06 06:39:15.913 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:18
2023-06-06 06:39:16.932 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:19
2023-06-06 06:39:18.015 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:20
2023-06-06 06:39:19.099 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:21
2023-06-06 06:39:20.229 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:22
2023-06-06 06:39:21.301 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:24
2023-06-06 06:39:22.342 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-06 06:39:56.232 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:58
2023-06-06 06:39:57.249 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:59
2023-06-06 06:39:58.270 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:01
2023-06-06 06:39:59.288 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:02
2023-06-06 06:40:00.307 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:03
2023-06-06 06:40:01.396 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:04
2023-06-06 06:40:02.493 | INFO     | pip

2023-06-06 06:40:34.944 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:37
2023-06-06 06:40:35.967 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:38
2023-06-06 06:40:37.000 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:39
2023-06-06 06:40:38.026 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:40
2023-06-06 06:40:39.061 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:41
2023-06-06 06:40:40.077 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:42
2023-06-06 06:40:41.094 | INFO     | pip

...

2023-06-06 06:46:14.763 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:17
2023-06-06 06:46:15.779 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:18
2023-06-06 06:46:16.795 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:19
2023-06-06 06:46:17.814 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:20
2023-06-06 06:46:18.829 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:21
2023-06-06 06:46:19.844 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:22
2023-06-06 06:46:20.861 | INFO     | pip

2023-06-06 06:46:52.509 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:55
2023-06-06 06:46:53.524 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:56
2023-06-06 06:46:54.547 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:57
2023-06-06 06:46:55.562 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:58
2023-06-06 06:46:56.576 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:59
2023-06-06 06:46:57.591 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:00
2023-06-06 06:46:58.612 | INFO     | pip

2023-06-06 06:47:30.241 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:32
2023-06-06 06:47:31.264 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:33
2023-06-06 06:47:32.282 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:35
2023-06-06 06:47:33.297 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:36
2023-06-06 06:47:34.313 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:37
2023-06-06 06:47:35.334 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:38
2023-06-06 06:47:36.370 | INFO     | pip

2023-06-06 06:48:08.122 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:10
2023-06-06 06:48:09.137 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:11
2023-06-06 06:48:10.152 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:12
2023-06-06 06:48:11.192 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:13
2023-06-06 06:48:12.213 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:14
2023-06-06 06:48:13.228 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:15
2023-06-06 06:48:14.244 | INFO     | pip

2023-06-06 06:48:45.931 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:48
2023-06-06 06:48:46.951 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:49
2023-06-06 06:48:49.022 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:89 - Job is success!!! Job id is 202306060637569089540
2023-06-06 06:48:49.023 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:90 - Total time: 0:10:51


當訓練結束後, 模型會用來做預測. 使用者可以自由選擇要不要儲存此次 `pipeline` 以方便未來重複使用
使用 `pipeline.dump(pipeline_saved_path)` 來完成儲存

In [11]:
pipeline.dump("pipeline_saved/hetero_two_party_hybrid_input_classification_secure_boost_tree.pkl");

### 建構 Inference pipeline 範例

首先, 使用 `PipeLine.load_model_from_file` load `pkl` 檔

部署 Inference 需要的模塊, 在這邊是 `data_transform_0`, `onehot_encoder_0`, `intersect_0`, `hetero_secureboost_0`

In [12]:
pipeline = PipeLine.load_model_from_file('pipeline_saved/hetero_two_party_hybrid_input_classification_secure_boost_tree.pkl')
pipeline.deploy_component([pipeline.data_transform_0, 
                           pipeline.onehot_encoder_0, 
                           pipeline.intersect_0, 
                           pipeline.hetero_secureboost_0]);

接著, 部署 `Reader` 模塊 `reader_1` 來讀取新data

In [13]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=guest).component_param(table={"name": "titanic_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=host).component_param(table={"name": "titanic_hetero_host", "namespace": "experiment"})

最後, 部署新的 `Evaluation` 來衡量 predict ( Inference ) 的表現

In [14]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

整合所有模塊

In [15]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


預測!

In [16]:
predict_pipeline.predict()

2023-06-06 06:48:52.696 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306060648521205430

2023-06-06 06:48:52.703 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-06-06 06:48:53.720 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-06 06:48:53.722 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:01
2023-06-06 06:48:54.739 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:02
2023-06-06 06:48:55.754 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:03
2023-06-06 06:48:56.769 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:04
2023-06-06 06:48:57.786 | INFO     

2023-06-06 06:49:31.293 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-06 06:49:32.321 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-06 06:49:33.351 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-06 06:49:34.375 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
2023-06-06 06:49:35.409 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:42
2023-06-06 06:49:36.430 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:43
2023-06-06 06:49:37.451 | INFO     | pipeline.utils.invoker.job_

2023-06-06 06:50:08.550 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:15
2023-06-06 06:50:09.566 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:16
2023-06-06 06:50:10.581 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:17
2023-06-06 06:50:11.599 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:18
2023-06-06 06:50:12.744 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:20
2023-06-06 06:50:13.813 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:21
2023-06-06 06:50:14.958 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-06 06:50:48.218 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-06-06 06:50:49.243 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-06-06 06:50:50.259 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-06-06 06:50:51.273 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:58
2023-06-06 06:50:52.288 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:59
2023-06-06 06:50:53.305 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:00
2023-06-06 06:50:54.331 | INFO     | pip

2023-06-06 06:51:26.861 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:34
2023-06-06 06:51:27.877 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:35
2023-06-06 06:51:28.909 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:36
2023-06-06 06:51:29.933 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:37
2023-06-06 06:51:31.162 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:38
2023-06-06 06:51:32.178 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:39
2023-06-06 06:51:33.194 | INFO     | pip

用 `pipeline.get_component('evaluation_0').get_summary()` 

來取得 `evaluation_0` 模塊的資訓儲並存成 json 檔

In [17]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/hetero_two_party_hybrid_input_classification_secure_boost_tree.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")

Write in metadata_saved_dir : /data/projects/fate/persistence/metadata/two_party_hybrid_input_binary_classification.json 
 {
    "hetero_secureboost_0": {
        "train": {
            "auc": 0.8822393719575198,
            "ks": 0.6337359792924936
        }
    }
}


# Hetero_NN

In [1]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "titanic_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/titanic_hetero_guest.csv")

In [2]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-06-12 03:07:45.946 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306120307458060740

2023-06-12 03:07:45.953 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


2023-06-12 03:07:46.962 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-12 03:07:47.976 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-12 03:07:47.977 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-12 03:07:48.992 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-12 03:07:50.006 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-12 03:07:51.020 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-06-12 03:07:52.033 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-06-12 03:07:5

#### 因為我們使用了 `OneHotEncoder ` , 要與 host 端溝通計算 onehot 完後會有幾個column, 方便設定模型參數 

In [ ]:
import torch as t
from torch import nn
from torch import optim

from collections import OrderedDict
from pipeline import fate_torch as ft
from pipeline import fate_torch_hook
from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import OneHotEncoder
from pipeline.component import Evaluation
from pipeline.component import HeteroNN
from pipeline.component import Intersection
from pipeline.component import Reader
from pipeline.interface import Data
from pipeline.interface import Model
from pipeline.utils.tools import load_job_config

# this is important, modify torch modules so that Sequential model be parsed by pipeline
fate_torch_hook(t)

pipeline = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=guest).component_param(
    table={"name": "titanic_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=host).component_param(
    table={"name": "titanic_hetero_host", "namespace": "experiment"})


data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(
    with_label=True, label_name="survived", label_type="int",
    missing_fill=True, missing_fill_method="designated", default_value=["O", "male", 29.7, 0], outlier_replace=False,
    data_type="float", exclusive_data_type={"pclass":"str", "sex":"str", "sibsp":"str"}
)

data_transform_0.get_party_instance(role='host', party_id=[host]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=["100", "0.0", "N"], outlier_replace=False,
    data_type="float", exclusive_data_type={"parch":"str", "embarked":"str"}
)

onehot_encoder_0 = OneHotEncoder(name="onehot_encoder_0")

onehot_encoder_0.get_party_instance(role='guest', party_id=guest).component_param(
    transform_col_indexes=[0, 1, 3], transform_col_names=["pclass", "sex", "sibsp"]
)

onehot_encoder_0.get_party_instance(role='host', party_id=[host]).component_param(
    transform_col_indexes=[0, 2], transform_col_names=["parch", "embarked"]
)

intersection_0 = Intersection(name="intersection_0")

# define network structure in torch style #
# define guest model
Linear = nn.Linear
ReLU = nn.ReLU
guest_bottom_a = Linear(13, 16, True)
seq = nn.Sequential(
    OrderedDict([
        ('layer_0', guest_bottom_a),
        ('relu_0', ReLU())
    ])
)

seq2 = nn.Sequential(
    ReLU(),
    Linear(16, 2, True),
) 

# define host model
seq3 = nn.Sequential(
    Linear(12, 16, True),
    ReLU(),
)

# use interactive layer after fate_torch_hook
interactive_layer = t.nn.InteractiveLayer(out_dim=16, guest_dim=16, host_dim=16, host_num=1)

# loss fun
ce_loss_fn = nn.CrossEntropyLoss()

# optimizer, after fate torch hook optimizer can be created without parameters
opt: ft.optim.Adam = optim.Adam(lr=1e-02)

hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=30, floating_point_precision=None,
                       interactive_layer_lr=1e-01, batch_size=512, early_stop="diff",
                       coae_param={'enable': True, 'epoch': 30})
guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
guest_nn_0.add_bottom_model(seq)
guest_nn_0.add_top_model(seq2)
host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)
host_nn_0.add_bottom_model(seq3)

hetero_nn_0.set_interactive_layer(interactive_layer)

hetero_nn_0.compile(opt, loss=ce_loss_fn)

hetero_nn_1 = HeteroNN(name="hetero_nn_1")
evaluation_0 = Evaluation(name="evaluation_0")

pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(onehot_encoder_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(intersection_0, data=Data(data=onehot_encoder_0.output.data))
pipeline.add_component(hetero_nn_0, data=Data(train_data=intersection_0.output.data))
pipeline.add_component(hetero_nn_1, data=Data(test_data=intersection_0.output.data),
                       model=Model(model=hetero_nn_0.output.model))
pipeline.add_component(evaluation_0, data=Data(data=hetero_nn_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-12 06:18:21.331 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306120618209154370

2023-06-12 06:18:21.339 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-12 06:18:22.348 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-12 06:18:23.363 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-12 06:18:23.365 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-06-12 06:18:24.380 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-06-12 06:18:25.396 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04


In [19]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/hetero_two_party_hybrid_input_classification_hetero_nn.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")

Write in metadata_saved_dir : /data/projects/fate/persistence/metadata/hetero_two_party_hybrid_input_classification_hetero_nn.json 
 {
    "hetero_nn_0": {
        "train": {
            "auc": 0.5080848752117088,
            "ks": 0.0805291918320391
        }
    }
}
